# Wake steering

This example demonstrates how to optimize the yaw angles of wind turbine rotors, such that the wakes are steered away from downwind turbines.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import foxes
import foxes.variables as FV
from iwopy.interfaces.pymoo import Optimizer_pymoo
from foxes_opt.problems import OptFarmVars
from foxes_opt.objectives import MaxFarmPower

Let's create the *foxes* engine, which will be used for all calculations:

In [ ]:
engine = foxes.Engine.new(
    "process",
    chunk_size_states=500,
    chunk_size_points=5000,
    verbosity=0,
)

In this example we are looking at a small 3 x 3 regualar wind farm with `NREL5MW` turbines:

In [ ]:
farm = foxes.WindFarm()
foxes.input.farm_layout.add_grid(
    farm,
    xy_base=np.array([500.0, 500.0]),
    step_vectors=np.array([[1300.0, 0], [200, 600.0]]),
    steps=(3, 3),
    turbine_models=["opt_yawm", "yawm2yaw", "NREL5MW"],
    verbosity=0,
)

ax = foxes.output.FarmLayoutOutput(farm).get_figure()
plt.show()
plt.close(ax.get_figure())

Notice how the name of the optimization model `opt_yawm`, which will be defined shortly, appears in the list of turbine models. The idea is as follows:

- First, the optimizer sets the `FV.YAWM` variable, representing _yaw misalignment_ in degrees, i.e., a delta yaw value from greedy conditions
- Then, the model `yawm2yaw` translates this into the absolute yaw value, i.e., the absolute turbine axis orientation, expressed in degrees
- This setting is then used for thrust and power calculations, by the turbine type model `NREL5MW`.

We are considering sinlge-state uniform inflow conditions:

In [ ]:
states = foxes.input.states.SingleStateStates(ws=9, wd=270, ti=0.06, rho=1.225)

The algorithm is defined next, including a TI wake model and the `Bastankhah2016` wake model. Also notice the wake frame choice `yawed`, which realizes the wake bending for yawed conditions:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    rotor_model="grid25",
    wake_models=["IECTI2019k_quadratic_ambka02", "Bastankhah2016_linear_ambka02"],
    wake_frame="yawed",
    verbosity=0,
)

The optimization problem is power maximization by finding the optimal value of the variable `FV.YAWM` for each turbine:

In [ ]:
problem = OptFarmVars("opt_yawm", algo)
problem.add_var(FV.YAWM, float, 0.0, -40.0, 40.0, level="turbine")
problem.add_objective(MaxFarmPower(problem))
problem.initialize()

Next, we setup the solver:

In [ ]:
solver = Optimizer_pymoo(
    problem,
    problem_pars=dict(vectorize=True),
    algo_pars=dict(type="GA", pop_size=100, seed=42),
    setup_pars=dict(),
    term_pars=("n_gen", 100),
)
solver.initialize()
solver.print_info()

Now everything is setup, and we can solve the problem:

In [ ]:
with engine:
    results = solver.solve()
    solver.finalize(results)

In [ ]:
print()
print(results)

fr = results.problem_results.to_dataframe()
fr[[FV.X, FV.Y, FV.AMB_WD, FV.YAWM, FV.TI, FV.REWS, FV.P]]

Finally, we can visualize the result by looking at the flow field:

In [ ]:
with engine:
    o = foxes.output.FlowPlots2D(algo, results.problem_results)
    plot_data = o.get_mean_data_xy("WS", resolution=10, xmax=5000)

fig = o.get_mean_fig_xy(plot_data)
plt.show()

Clearly the turbines are trying to avoid hitting downwind turbines with their wakes.